
<div style="text-align: center;">
  <img src="https://github.com/Hack-io-Data/Imagenes/blob/main/01-LogosHackio/logo_amarillo@4x.png?raw=true" alt="esquema" />
</div>


# Laboratorio ETL: Análisis del Sistema Energético en España

## Objetivo

Durante todos los laboratorios de esta semana realizarás un proceso completo de ETL para analizar la relación entre la demanda, el consumo y la generación eléctrica en diferentes provincias de España a lo largo de un año. Además, complementarán este análisis con datos demográficos y económicos extraídos del Instituto Nacional de Estadística (INE). El **objetivo principal** del análisis es **examinar cómo la demanda, el consumo y la generación eléctrica en diferentes provincias de España a lo largo de los años están influenciados por factores demográficos y económicos, como la población y el PIB provincial**. El análisis busca identificar patrones y correlaciones entre estas variables para comprender mejor las dinámicas energéticas regionales y su relación con el desarrollo socioeconómico en España.

Antes de realizar el análisis, vamos a definir las hipótesis con las que vamos a trabajar, las cuales definirán todo tu análisis y planteamiento de los laboratorios: 

- **Hipótesis 1: La demanda eléctrica está correlacionada con la población de la provincia.** Provincias con mayor población tienden a tener una mayor demanda eléctrica.
  
- **Hipótesis 2: El crecimiento económico (medido por el PIB) está correlacionado con la demanda eléctrica.** Las provincias con un PIB más alto o en crecimiento experimentan una mayor demanda de energía.

- **Hipótesis 3: La proporción de generación renovable está relacionada con factores económicos o geográficos.** Provincias con un mayor desarrollo económico o con condiciones geográficas favorables (como más horas de sol o viento) tienden a generar más energía renovable.


## Tareas Laboratorio Extracción

En el laboratorio de hoy tendrás que extraer la información necesaria para obtener tu objetivo de las siguientes fuentes de datos (deberás usar API's y herramientas de *web scrapping*):

- **Datos de la API de Red Eléctrica Española (REE):** Deberás extraer datos mensuales a nivel comunidad_autónoma de los siguientes aspectos:

  - **Demanda Eléctrica:** Nos proporciona los datos de demanda eléctrica a nivel provincial, agregados de manera mensual. Tendrás que usar el endpoint de "https://apidatos.ree.es/es/datos/demanda/evolucion", añadiendo los parámetros que sean necesarios. 

  - **Generación Eléctrica:** Nos proporciona los datos de generación eléctrica a nivel provincial, diferenciando entre fuentes de energía (eólica, solar, hidroeléctrica, etc.), agregados de manera mensual. Tendrás que usar el endpoint de "https://apidatos.ree.es/es/datos/generacion/estructura-renovables", añadiendo los parámetros que sean necesarios.

  La documentación de la API la encontrarás en [este link](https://www.ree.es/es/apidatos). Recuerda leer en detenimiento la documentación. 

- **Datos del Instituto Nacional de Estadística (INE):** Además de los datos de la REE, debes extraer y utilizar datos socioeconómicos de las siguientes páginas del INE:

- **Datos Demográficos:** Extraer los datos de población por provincias, diferenciando por grupos de edad, sexo, y extrajeros. Estos datos serán utilizados para analizar cómo la población afecta a la demanda eléctrica en cada provincia.

  - **Página web:** [INE - Población por provincias](https://www.ine.es/dyngs/INEbase/es/operacion.htm?c=Estadistica_C&cid=1254736177012&menu=resultados&idp=1254734710990)

  - "Principales series 1998-2022" --> "Por provincia" --> " Población por provincias, edad (3 grupos de edad), españoles/Extranjeros, Sexo y año"

- **Datos Económicos:**

  - **Página web:** [INE - PIB por provincias](https://www.ine.es/dynt3/inebase/es/index.htm?padre=10426&capsel=10429). 

  - **Pasos para la extracción**:" Resultados provinciales. Serie contable 2016-2021" --> "P.I.B. a precios de mercado y valor añadido bruto a precios básicos por ramas de actividad: Precios corrientes por provincias y periodo."



NOTA1: Tienes que sacar muchos datos, pero recuerda que hemos aprendido herramientas de asincronia que te pueden ayudar en este paso de la ETL. 

NOTA2: Todos estos datos los debes sacar para los años 2019-2020-2021

In [130]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from time import sleep
# Selenium para establecer la configuración del driver
# -----------------------------------------------------------------------
from selenium import webdriver

# Para generar una barra de proceso en los bucles for
# -----------------------------------------------------------------------
from tqdm import tqdm

# Para trabajar con ficheros
# -----------------------------------------------------------------------
import os
import requests
import re
import json
import zipfile
import shutil


# Crear directorios 

In [77]:
ruta_datos_y_datosdescargados = "datos/DatosDescargados"
os.makedirs(ruta_datos_y_datosdescargados,exist_ok=True)


# Extraer INE por provincia (Población)

In [78]:
# lo primero que vamos a hacer es configurar nuestras preferencias del navegador para el driver
chrome_options = webdriver.ChromeOptions()

# establacemos las preferencias que queremos
prefs = {
    
    "download.default_directory": "C:\\Users\\ruipe\\Desktop\\Data Science\\Semana 5\\labs\\laboratorio 1\\datos\\DatosDescargados\\",  # AQUÍ CADA UNO TENDREMOS QUE PONER LA RUTA QUE QUERAMOS PARA QUE SE GUARDEN LOS ARCHIVOS DESCARGADOS
    "download.prompt_for_download": False,   # desactiva el diálogo que Chrome normalmente muestra para pedir confirmación del usuario antes de descargar un archivo
    "directory_upgrade": True,    # hace que Chrome actualice el directorio de descarga predeterminado a la nueva ubicación especificada por download.default_directory si esta ha cambiado.
}

url = "https://www.ine.es/dyngs/INEbase/es/operacion.htm?c=Estadistica_C&cid=1254736177012&menu=resultados&idp=1254734710990"
chrome_options.add_experimental_option("prefs", prefs)

driver = webdriver.Chrome(options=chrome_options)
driver.get(url)
driver.maximize_window()
sleep(1)
driver.find_element("css selector","#aceptarCookie").click()
sleep(1)
#Hacer click en Provincias
tabla = driver.find_element("css selector","#cuerpo")
subtabla = tabla.find_element("css selector", "#tabs-1254736195461 > div:nth-child(2) > div.cuerpo_centro_principal_IBresul")
opciones = subtabla.find_element("xpath","/html/body/div/main/section[2]/div[1]/div[1]/div[1]/ul/li")
opciones.find_element("xpath","/html/body/div/main/section[2]/div[1]/div[1]/div[1]/ul/li/ul/li[3]/a").click()
sleep(1)
#Hacer scroll y click en Población por provincias, edad (3 grupos de edad), españoles/Extranjeros, Sexo y año
driver.execute_script("window.scrollTo(0, 200)")
tabla_provincias = driver.find_element("xpath","/html/body/div/main/div[2]/ul/li[4]")
tabla_provincias.find_element("xpath","/html/body/div/main/div[2]/ul/li[4]/ul/li[1]/a").click()
sleep(1)
#Hacer click en todo primera tabla
body = driver.find_element("xpath","/html/body/div/main/form")
opciones_descarga = body.find_element("xpath","/html/body/div/main/form/ul")
tabla1 = opciones_descarga.find_element("xpath","/html/body/div[1]/main/form/ul/li[1]/ul/li[1]/div/fieldset")
tabla1_seccion=tabla1.find_element("xpath","/html/body/div/main/form/ul/li[1]/ul/li[1]/div/fieldset/div[2]")
tabla1_seccion.find_element("xpath","/html/body/div/main/form/ul/li[1]/ul/li[1]/div/fieldset/div[2]/button[1]").click()

#Hacer click en todo segunda tabla
tabla2 = opciones_descarga.find_element("xpath","/html/body/div/main/form/ul/li[1]/ul/li[2]/div/fieldset")
tabla2_seccion = tabla2.find_element("xpath","/html/body/div/main/form/ul/li[1]/ul/li[2]/div/fieldset/div[2]")
tabla2_seccion.find_element("xpath","/html/body/div/main/form/ul/li[1]/ul/li[2]/div/fieldset/div[2]/button[1]").click()

#Hacer click en todo tercera tabla
tabla3 = opciones_descarga.find_element("xpath","/html/body/div/main/form/ul/li[1]/ul/li[3]/div/fieldset")  
tabla3_seccion = tabla3.find_element("xpath","/html/body/div/main/form/ul/li[1]/ul/li[3]/div/fieldset/div[2]")
tabla3_seccion.find_element("xpath","/html/body/div/main/form/ul/li[1]/ul/li[3]/div/fieldset/div[2]/button[1]").click()

#Hacer click en todo cuarta tabla
tabla4 = opciones_descarga.find_element("xpath","/html/body/div/main/form/ul/li[1]/ul/li[4]/div/fieldset")  
tabla4_seccion = tabla4.find_element("xpath","/html/body/div/main/form/ul/li[1]/ul/li[4]/div/fieldset/div[2]")
tabla4_seccion.find_element("xpath","/html/body/div/main/form/ul/li[1]/ul/li[4]/div/fieldset/div[2]/button[1]").click()

#Hacer click en año 2019 tabla
tabla5 = opciones_descarga.find_element("xpath","/html/body/div/main/form/ul/li[1]/ul/li[5]/div/fieldset")
tabla5.find_element("xpath","/html/body/div/main/form/ul/li[1]/ul/li[5]/div/fieldset/select/option[1]").click()
tabla5.find_element("xpath","/html/body/div/main/form/ul/li[1]/ul/li[5]/div/fieldset/select/option[2]").click()
tabla5.find_element("xpath","/html/body/div/main/form/ul/li[1]/ul/li[5]/div/fieldset/select/option[3]").click()
tabla5.find_element("xpath","/html/body/div/main/form/ul/li[1]/ul/li[5]/div/fieldset/select/option[4]").click()

body.find_element("xpath","/html/body/div/main/form/div[5]/input[3]").click()

# Hacer click en descargar
sleep(2)
driver.find_element("xpath","/html/body/div[1]/main/ul/li/div/div/form[2]/button").click()
sleep(1)

select_download = WebDriverWait(driver,40).until(EC.presence_of_element_located(('xpath','/html/body/div[6]/article/div/div/iframe')))
driver.switch_to.frame(select_download)
sleep(2)
try:
    driver.find_element("xpath", "/html/body/form/ul/li[4]/label").click()
    print("Encontrado")
except:
    print("Cagaste")
sleep(2)
driver.quit()

Encontrado


In [84]:
old_archivo = os.listdir(ruta_datos_y_datosdescargados)[0]
new_archivo = "DatosInePorProvincias2019-2021.csv"
os.rename(f"{ruta_datos_y_datosdescargados}/{old_archivo}",f"{ruta_datos_y_datosdescargados}/{new_archivo}")

# Extraer PIB por Provincia

In [97]:
# lo primero que vamos a hacer es configurar nuestras preferencias del navegador para el driver
chrome_options = webdriver.ChromeOptions()

# establacemos las preferencias que queremos
prefs = {
    
    "download.default_directory": "C:\\Users\\ruipe\\Desktop\\Data Science\\Semana 5\\labs\\laboratorio 1\\datos\\DatosDescargados\\",  # AQUÍ CADA UNO TENDREMOS QUE PONER LA RUTA QUE QUERAMOS PARA QUE SE GUARDEN LOS ARCHIVOS DESCARGADOS
    "download.prompt_for_download": False,   # desactiva el diálogo que Chrome normalmente muestra para pedir confirmación del usuario antes de descargar un archivo
    "directory_upgrade": True,    # hace que Chrome actualice el directorio de descarga predeterminado a la nueva ubicación especificada por download.default_directory si esta ha cambiado.
}

url = "https://www.ine.es/dynt3/inebase/es/index.htm?padre=10426&capsel=10429"
chrome_options.add_experimental_option("prefs", prefs)

driver = webdriver.Chrome(options=chrome_options)
driver.get(url)
driver.maximize_window()
sleep(1)
driver.find_element("css selector","#aceptarCookie").click()
sleep(1)
# Hacer click en el enlace 
main = driver.find_element("xpath","/html/body/div/main")
tabla = main.find_element("xpath","/html/body/div/main/div[2]/ul")
tree = tabla.find_element("xpath","/html/body/div/main/div[2]/ul/li[3]")
ul = tree.find_element("xpath","/html/body/div/main/div[2]/ul/li[3]/ul")
ul.find_element("xpath","/html/body/div/main/div[2]/ul/li[3]/ul/li[2]/a").click()

#Hacer click en todo primera tabla
body = driver.find_element("xpath","/html/body/div[1]/main/form")
opciones_descarga = body.find_element("xpath","/html/body/div/main/form/ul/li[1]/ul/li[2]/div/fieldset")
tabla1 = opciones_descarga.find_element("xpath","/html/body/div/main/form/ul/li[1]/ul/li[2]/div/fieldset/div[2]")
opciones_descarga.find_element("xpath","/html/body/div/main/form/ul/li[1]/ul/li[2]/div/fieldset/div[2]/button[1]").click()

#Hacer click en todo segunda tabla
opciones_descarga = body.find_element("xpath","/html/body/div/main/form/ul/li[1]/ul/li[3]/div/fieldset")
tabla2 = opciones_descarga.find_element("xpath","/html/body/div/main/form/ul/li[1]/ul/li[3]/div/fieldset/div[2]")
opciones_descarga.find_element("xpath","/html/body/div/main/form/ul/li[1]/ul/li[3]/div/fieldset/div[2]/button[1]").click()

#Hacer click en 2019,2020 y 2021
opciones_descarga = body.find_element("xpath","/html/body/div/main/form/ul/li[1]/ul/li[3]/div/fieldset")
tabla3 = opciones_descarga.find_element("xpath","/html/body/div/main/form/ul/li[1]/ul/li[4]/div/fieldset/select")
tabla3.find_element("xpath","/html/body/div/main/form/ul/li[1]/ul/li[4]/div/fieldset/select/option[2]").click()
tabla3.find_element("xpath","/html/body/div/main/form/ul/li[1]/ul/li[4]/div/fieldset/select/option[3]").click()
tabla3.find_element("xpath","/html/body/div/main/form/ul/li[1]/ul/li[4]/div/fieldset/select/option[4]").click()

#click en Buscar
body.find_element("xpath","/html/body/div/main/form/div[5]/input[3]").click()
sleep(2)

# Click en descargar
driver.find_element("xpath","/html/body/div[1]/main/ul/li/div/div/form[2]/button").click()
sleep(1)
#Esperar que aparezca el iframe
select_download = WebDriverWait(driver,40).until(EC.presence_of_element_located(('xpath','/html/body/div[6]/article/div/div/iframe')))
driver.switch_to.frame(select_download)
sleep(2)
try:
    driver.find_element("xpath", "/html/body/form/ul/li[4]/label").click()
    print("Encontrado")
except:
    print("Cagaste")
sleep(2)
driver.quit()


Encontrado


In [101]:
old_archivo = os.listdir(ruta_datos_y_datosdescargados)[0]
new_archivo = "DatosPIBPorProvincias2019-2021.csv"
os.rename(f"{ruta_datos_y_datosdescargados}/{old_archivo}",f"{ruta_datos_y_datosdescargados}/{new_archivo}")

# API Red Eléctrica
 

In [132]:
cod_comunidades = {'Ceuta': 8744,
                        'Melilla': 8745,
                        'Andalucía': 4,
                        'Aragón': 5,
                        'Cantabria': 6,
                        'Castilla - La Mancha': 7,
                        'Castilla y León': 8,
                        'Cataluña': 9,
                        'País Vasco': 10,
                        'Principado de Asturias': 11,
                        'Comunidad de Madrid': 13,
                        'Comunidad Foral de Navarra': 14,
                        'Comunitat Valenciana': 15,
                        'Extremadura': 16,
                        'Galicia': 17,
                        'Illes Balears': 8743,
                        'Canarias': 8742,
                        'Región de Murcia': 21,
                        'La Rioja': 20}

In [133]:
years = [2019,2020,2021]
ruta_years_demanda="datos/DatosDescargados/RedElectrica/EvolucionDemanda"
os.makedirs(ruta_years_demanda,exist_ok=True)
ruta_years_generacion_electrica="datos/DatosDescargados/RedElectrica/GeneracionElectrica"
os.makedirs(ruta_years_generacion_electrica,exist_ok=True)
for year in years:
    routedemanda = f"{ruta_years_demanda}/{year}"
    os.makedirs(routedemanda,exist_ok=True)
    routegenelectric = f"{ruta_years_generacion_electrica}/{year}"
    os.makedirs(routegenelectric,exist_ok=True)

### Demanda
- https://apidatos.ree.es/es/datos/demanda/evolucion?start_date={year}-01-01T00:00&end_date={year}-12-31T23:59&time_trunc=month&geo_limit=ccaa&geo_ids={codigo}

In [145]:
for comunidad,codigo in tqdm(cod_comunidades.items()):
    for year in years:
        
        url = f"https://apidatos.ree.es/es/datos/demanda/evolucion?start_date={year}-01-01T00:00&end_date={year}-12-31T23:59&time_trunc=month&geo_limit=ccaa&geo_ids={codigo}"  
        response = requests.get(url)

        ruta_to_save = f"{ruta_years_demanda}/{year}/demanda_{comunidad}_{year}.json"
        # Verificar si la solicitud fue exitosa
        if response.status_code == 200:
            datos = response.json()  # Convertir la respuesta en JSON
            # Guardar los datos JSON en un archivo
            with open(ruta_to_save, 'w') as f:
                json.dump(datos, f, indent=4)
        else:
            print(f"Error en la solicitud: {response.status_code},{comunidad}:{codigo}")     

100%|██████████| 19/19 [03:41<00:00, 11.66s/it]


### Generación Eléctrica
- https://apidatos.ree.es/es/datos/generacion/estructura-renovables

In [146]:
for comunidad,codigo in tqdm(cod_comunidades.items()):
    for year in years:
        
        url = f"https://apidatos.ree.es/es/datos/generacion/estructura-renovables?start_date={year}-01-01T00:00&end_date={year}-12-31T23:59&time_trunc=month&geo_limit=ccaa&geo_ids={codigo}"  
        response = requests.get(url)

        ruta_to_save = f"{ruta_years_generacion_electrica}/{year}/generacion_electrica_{comunidad}_{year}.json"
        # Verificar si la solicitud fue exitosa
        if response.status_code == 200:
            datos = response.json()  # Convertir la respuesta en JSON
            # Guardar los datos JSON en un archivo
            with open(ruta_to_save, 'w') as f:
                json.dump(datos, f, indent=4)
        else:
            print(f"Error en la solicitud: {response.status_code},{comunidad}:{codigo}")     

  0%|          | 0/19 [00:00<?, ?it/s]

Error en la solicitud: 502,Ceuta:8744
Error en la solicitud: 502,Ceuta:8744


  5%|▌         | 1/19 [00:07<02:17,  7.64s/it]

Error en la solicitud: 502,Ceuta:8744


100%|██████████| 19/19 [01:48<00:00,  5.72s/it]
